 ## **Importando e instalando libs necessárias**

In [ ]:
!pip install -qU langchain-groq
!pip install -U langchain-community
!pip install faiss-cpu
!pip install langchain-openai

In [74]:
from google.colab import userdata
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import getpass
import os

## **Criando meu webscrapper com o embedder da Hugging face**

In [75]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

def url_to_vector(url):
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

  loader = WebBaseLoader(url)
  docs = loader.load()

  text_splitter = RecursiveCharacterTextSplitter()
  documents = text_splitter.split_documents(docs)
  vector = FAISS.from_documents(documents, embeddings)
  retriever = vector.as_retriever()
  return retriever

In [76]:

os.environ["GROQ_API_KEY"] = userdata.get('Croq')

from langchain_groq import ChatGroq

llm = ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct")

In [77]:
#Testando meu prompt sem apoio dos dados do RAG

def oscar(filme, ano, llm):
  prompt = PromptTemplate(
      input_variables= ['filme', 'ano'], #os dois parâmetros que ele vai aceitar são o filme e o ano
      template = "Quanto vendeu o filme {filme} em {ano}" #mudei o template do prompt para fazer uns testes
  )

  oscar_chain = LLMChain(llm=llm, prompt= prompt)

  response = oscar_chain({'filme': filme, 'ano': ano})

  return response

In [78]:
if __name__=="__main__":
  response = oscar('Lilo and Stich live action', 2025, llm) # Tentei modificar para um filme mais atual para testar se ele reconhece
  print(response['text'])

Desculpe, mas não tenho informações sobre um filme live-action de "Lilo & Stitch" lançado em 2025. Até a data do meu último conhecimento em abril de 2023, não havia anúncios oficiais sobre um filme live-action de "Lilo & Stitch" programado para 2025.

No entanto, é importante notar que a Disney tem planos de produzir live-actions de muitos de seus filmes clássicos e populares. "Lilo & Stitch" é uma franquia bem querida e não seria surpreendente se um filme live-action fosse anunciado no futuro.

Se você está procurando informações sobre vendas de bilheteria, recomendo verificar fontes como Box Office Mojo ou The Numbers para obter informações atualizadas sobre filmes lançados recentemente.

Caso você tenha mais alguma informação ou contexto sobre o filme que está se referindo, ficarei feliz em tentar ajudar!


## **Testando meu modelo agora com o apoio do RAG com o site da wikipedia**

In [81]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente útil. Precisa ser bem curto nas respostas"),
    ("human", "Contexto:\n{context}\n\nPergunta: {input}") #Meu prompt que aceita um contexto e o input
])

document_chain = create_stuff_documents_chain(llm, prompt)
retriever = url_to_vector("https://en-m-wikipedia-org.translate.goog/wiki/Lilo_&_Stitch_(2025_film)?_x_tr_sl=en&_x_tr_tl=pt&_x_tr_hl=pt&_x_tr_pto=tc") #URL do filme live action do Lilo and Stich
retriever_chain = create_retrieval_chain(retriever, document_chain)
response = retriever_chain.invoke({"input":"Quanto vendeu o filme live action do Lilo and Stich em 2025?"})
print(response['answer'])

#Como foi possível ver, com o RAG agora é possível pergutar sobre um filme mais atual, um que o modelo não tem na sua própria base de dados.
#A RAG providencia uma base de dados nova para o LLM se tornando bastante útil.

O filme "Lilo & Stitch" live-action de 2025 arrecadou um total mundial de $994.3 milhões. Desse total, $414.6 milhões foram provenientes dos Estados Unidos e Canadá, e $579.7 milhões de outros territórios.
